In [1]:
import os
import gc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf

In [2]:
# Global Variables
BATCH_SIZE = 16
IMG_SIZE = 128

In [3]:
# Paths: For local machine
train_dir = 'C:/Users/Nick/Documents/SchoolStuff/spring2024/machineLearning/final_project/tradingpatterns/stock_patterns/train'
validation_dir = 'C:/Users/Nick/Documents/SchoolStuff/spring2024/machineLearning/final_project/tradingpatterns/stock_patterns/validation'

# Create training and validation dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            color_mode='rgb',
                                                            image_size=(IMG_SIZE, IMG_SIZE))

validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 color_mode='rgb',
                                                                 image_size=(IMG_SIZE, IMG_SIZE))
class_names = train_dataset.class_names

Found 6404 files belonging to 4 classes.
Found 5984 files belonging to 4 classes.


In [4]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)
preprocess_input = tf.keras.applications.resnet.preprocess_input

In [5]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (IMG_SIZE, IMG_SIZE) + (3,)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                            include_top=False,
                                            weights='imagenet')

In [6]:
# This feature extractor converts each 255x255 image into a 5x5x1280 block of features
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)

In [7]:
base_model.trainable = False

# Adding a classification head
global_max_layer = tf.keras.layers.GlobalMaxPooling2D()
feature_batch_max = global_max_layer(feature_batch)

In [8]:
# Add a dense layer to convert it to a single prediction per image
# Test relu, leaky_relu, and sigmoid
prediction_layer = tf.keras.layers.Dense(len(class_names))
prediction_batch = prediction_layer(feature_batch_max)

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# Add the preprocessing layer
x = preprocess_input(inputs)
# Add the base model after, keeping training to false
x = base_model(x, training=False)
# Add max pooling layer
x = global_max_layer(x)
# Add dropout layer
x = tf.keras.layers.Dropout(0.2)(x)
# Add softmax prediction layer
outputs = prediction_layer(x)
# Create a model from inputs, outputs.
model = tf.keras.Model(inputs, outputs)

# We use binary accuracy here because of the Sigmoid?
# If we use a different activator then I am not sure 
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
initial_epochs = 10
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    steps_per_epoch=10,
                    validation_data=validation_dataset,
                    validation_steps=10)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 460ms/step - accuracy: 0.2378 - loss: 3.3591 - val_accuracy: 0.3375 - val_loss: 2.5440
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 274ms/step - accuracy: 0.2575 - loss: 3.3779 - val_accuracy: 0.3375 - val_loss: 2.6073
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 271ms/step - accuracy: 0.3021 - loss: 3.3566 - val_accuracy: 0.3313 - val_loss: 2.4653
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 276ms/step - accuracy: 0.3127 - loss: 3.4075 - val_accuracy: 0.2688 - val_loss: 2.2749
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 276ms/step - accuracy: 0.2646 - loss: 3.2409 - val_accuracy: 0.2375 - val_loss: 2.5403
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 269ms/step - accuracy: 0.2704 - loss: 3.2797 - val_accuracy: 0.3187 - val_loss: 2.0563
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 273ms/step - accuracy: 0.2858 - loss: 3.2335 - val_accuracy: 0.3875 - val_loss: 2.0473
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 269ms/step - accuracy: 0.2970 - loss: 3.0068 - val_accuracy: 0.

In [10]:
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# 
# loss = history.history['loss']
# val_loss = history.history['val_loss']
# 
# plt.figure(figsize=(8, 8))
# plt.subplot(2, 1, 1)
# plt.plot(acc, label='Training Accuracy')
# plt.plot(val_acc, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.ylabel('Accuracy')
# plt.ylim([0, .5])
# plt.title('Training and Validation Accuracy')
# 
# plt.subplot(2, 1, 2)
# plt.plot(loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.ylabel('Cross Entropy')
# plt.title('Training and Validation Loss')
# plt.xlabel('epoch')
# plt.ylim([0, 10])
# plt.show()

In [11]:
base_model.trainable = True

print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 120

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  175


In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=len(history.epoch),
                         steps_per_epoch=10,
                         validation_data=validation_dataset,
                         validation_steps=10)

Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 629ms/step - accuracy: 0.2245 - loss: 4.8901 - val_accuracy: 0.2500 - val_loss: 2.4724
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 448ms/step - accuracy: 0.2747 - loss: 4.4891 - val_accuracy: 0.3125 - val_loss: 2.2494
Epoch 13/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 446ms/step - accuracy: 0.2741 - loss: 4.4377 - val_accuracy: 0.2750 - val_loss: 2.6565
Epoch 14/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 446ms/step - accuracy: 0.3273 - loss: 3.7299 - val_accuracy: 0.2313 - val_loss: 2.8677
Epoch 15/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 446ms/step - accuracy: 0.2466 - loss: 4.4177 - val_accuracy: 0.3438 - val_loss: 2.7528
Epoch 16/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 446ms/step - accuracy: 0.1994 - loss: 4.0538 - val_accuracy: 0.3313 - val_loss: 2.6583
Epoch 17/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 448ms/step - accuracy: 0.2818 - loss: 3.9344 - val_accuracy: 0.3125 - val_loss: 2.5408
Epoch 18/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 449ms/step - accuracy: 0.2733 - loss: 4.4648 - val_acc

In [14]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 139ms/step - accuracy: 0.2989 - loss: 2.6542
Test accuracy : 0.2922297418117523


In [15]:
# acc += history_fine.history['accuracy']
# val_acc += history_fine.history['val_accuracy']
# 
# loss += history_fine.history['loss']
# val_loss += history_fine.history['val_loss']
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# 
# loss = history.history['loss']
# val_loss = history.history['val_loss']
# 
# plt.figure(figsize=(8, 8))
# plt.subplot(2, 1, 1)
# plt.plot(acc, label='Training Accuracy')
# plt.plot(val_acc, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.ylabel('Accuracy')
# plt.ylim([0, 1])
# plt.title('Training and Validation Accuracy')
# 
# plt.subplot(2, 1, 2)
# plt.plot(loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.ylabel('Cross Entropy')
# plt.title('Training and Validation Loss')
# plt.xlabel('epoch')
# plt.ylim([0, 10])
# plt.show()

In [16]:
# Add a softmax layer to our model and apply our test data
# probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
# image_batch, label_batch = test_dataset.as_numpy_iterator().next()
# predictions = probability_model.predict(image_batch)

In [17]:
# plt.figure(figsize=(10, 10))
# for i in range(9):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].astype("uint8"))
#   guess = str(class_names[np.argmax(predictions[i+9])])
#   actual = str(class_names[label_batch[i+9]])
#   title = "Prediction: " + guess + "\n" + "Actual: " + actual
#   plt.title(title)
#   plt.axis("off")